In [ ]:
from qiskit import QuantumCircuit, transpile, execute, IBMQ
from states import Key, Ciphertext
from global_parameters import GlobalParameters
from encryption_circuit import encrypt
from verification_circuit import verify_deletion_counts
from decryption_circuit import create_decryption_circuit, decrypt_results
from deletion_circuit import delete
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeMontreal
from utils import export_counts, random_bit_string
import os
import shutil

In [ ]:
# Qiskit parameters
provider = IBMQ.load_account()
noise_model = FakeMontreal()
backend = AerSimulator.from_backend(noise_model)
# backend = provider.get_backend("ibmq_santiago")
optimization_level = 1
shots = 1000

# Create data directory
if not os.path.exists("data"):
    os.mkdir("data")

In [ ]:
# Utility function to run a circuit, and return the result counts
def run_and_measure(circuit: QuantumCircuit, draw: bool = False) -> dict[str, int]:
    """Transpiles and runs the QuantumCircuit, and returns the resulting counts of the execution."""
    if draw:
        print(circuit.draw())
    transpiled_circuit = transpile(
        circuit, backend=backend, optimization_level=0)
    result = backend.run(transpiled_circuit, shots=shots).result()
    counts = result.get_counts()
    # Reverse the string since the most significant qubit is at the 0th index of the measurement string
    reversed_keys_dict = {}
    for key, value in counts.items():
        reversed_keys_dict[key[::-1]] = value
    return reversed_keys_dict

In [ ]:
# Encrypt 
global_params = GlobalParameters.generate_from_lambda(1)
key = Key.generate_key(global_params)
message = random_bit_string(global_params.n)
ciphertext = encrypt(message, key, global_params)
original_circuit = ciphertext.circuit.copy() # We might modify this circuit in subsequent steps

with open("data/global_params.txt", "w") as f:
    f.write(global_params.to_json())

with open("data/key.txt", "w") as f:
    f.write(key.to_json())

with open("data/ciphertext.txt", "w") as f:
    f.write(ciphertext.to_json())

with open("data/message.txt", "w") as f:
    f.write(message)

In [ ]:
# Test 1 - delete
deletion_circuit = delete(ciphertext)
deletion_counts = run_and_measure(deletion_circuit)
export_counts(deletion_counts, csv_filename="data/test1-deletion-counts.csv", key_label="Deletion measurement")
verify_deletion_counts(deletion_counts, key, global_params)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [ ]:
# Test 2 - decrypt
decryption_circuit = create_decryption_circuit(key, ciphertext)
decryption_counts = run_and_measure(decryption_circuit)
export_counts(decryption_counts, csv_filename="data/test2-decryption-counts.csv", key_label="Decryption measurement")
decrypt_results(decryption_counts, key, ciphertext, message)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [ ]:
# Test 3 - delete and then decrypt
deletion_circuit = delete(ciphertext)
ciphertext.circuit = deletion_circuit # To use the new circuit as the starting point of the decryption circuit
decryption_circuit = create_decryption_circuit(key, ciphertext)
counts = run_and_measure(decryption_circuit)
deletion_counts = {}
decryption_counts = {}
for measurement, count in counts.items():
    # Qiskit will return a space-separated string of the two measurements. 
    # run_and_measure will reverse the string so that the first substring is the first measurement.
    deletion, decryption = measurement.split(" ")
    deletion_counts[deletion] = deletion_counts.get(deletion, 0) + count
    decryption_counts[decryption] = decryption_counts.get(decryption, 0) + count

export_counts(deletion_counts, csv_filename="data/test3-deletion-counts.csv", key_label="Deletion measurement")
export_counts(decryption_counts, csv_filename="data/test3-decryption-counts.csv", key_label="Decryption measurement")
verify_deletion_counts(deletion_counts, key, global_params)
decrypt_results(decryption_counts, key, ciphertext, message)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [ ]:
# Create zip file
import shutil
shutil.make_archive("data_export", "zip", "data")